# Example 0: Convert external data

In this example we will download external data that will be used to estimate relative moment tensors for seismic aftershocks related to the 2015-2017 Pamir earthquake sequence and convert them to the data formats required by *relMT*. For background information on the data set, please refer to [Bloch et al. (2023, GJI)](https://doi.org/10.1093/gji/ggac473).

> **Note**
>
> This tutorial relies on the command line tools `cd` `mkdir` and `wget`.
>
> They are part of most operating systems, but not all.
>
> `head` and `ls` are also used, but not required.

We will start by importing modules required for this tutorial.

In [ ]:

from relmt import io, extra, mt, utils

## Download external data

We create a folder `ext/` to store the external data. For this example, we access the data publications

Bloch et al. (2022) Earthquake and Moment Tensor Catalogs of the 2015-2017 Pamir Earthquake Sequence. *GFZ Data Services*. https://doi.org/10.5880/fidgeo.2022.007

and

Bloch et al. (2026): Earthquake Phase Arrival Times of the 2015-2017 Pamir Earthquake Sequence. GFZ Data Services. https://doi.org/10.5880/fidgeo.2026.006

Let us create the directory, change into it and download the data.

In [ ]:
# Make the directory, no warning if exists
! mkdir -p ext data

# -nc avoids re-downloading, -nv is less verbose, -P sets target directory
! wget -nc -nv -P ext https://datapub.gfz.de/download/10.5880.FIDGEO.2022.007Geazgf/2022-007_Bloch-et-al_seismic_event_catalog.txt
! wget -nc -nv -P ext https://datapub.gfz.de/download/10.5880.FIDGEO.2022.007Geazgf/2022-007_Bloch-et-al_moment_tensor_catalog_correct_norm_v2.0.txt
! wget -nc -nv -P ext https://datapub.gfz.de/download/10.5880.FIDGEO.2026.006-Kacenu/2026-006_Bloch-et-al_2015-2017_pamir_catalog_bulletin.xml


## Create a *relMT* event file

We will now create a *relMT* compliant event file. Let us have a look at the provided catalog file.

In [ ]:

! head ext/2022-007_Bloch-et-al_seismic_event_catalog.txt

We find that the seismic event catalog contains the locations of the events in columns 7, 8 and 9 (0-based indexing), the origin time in column 6 (as a time stamp in seconds since 1970), and the magnitude in column 15. Some of the tools of the `extra` module will help us to:

* Convert the locations from geographic (degree) to UTM (meter) coordinates
* Provide a date string as an event name
* Save it in the `data/` subdirectory for use in the later examples

The Pamir highlands are located in [UTM zone 43 S](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system), in the Tajik Chinese border region.

In [ ]:
# The external seismic event catalog file
catf = "ext/2022-007_Bloch-et-al_seismic_event_catalog.txt"

# Geoconverter for UTM zone 43S (Pamir region)
def geoconv(lat, lon, dep):
    return extra.geoconverter_latlon2utm(lat, lon, dep, 43, "S")


# Event name is the origin time in YYYYMMDDHHMMSS format
def nameconv(timetuple):
    yr, mo, da, hr, mn, sc = timetuple
    return f"{yr}{mo}{da}{hr}{mn}{sc[:2]}"


# Read the seismic event catalog into an event dictionary
evd = io.read_ext_event_table(
    catf,
    *(8, 7, 9, 6, 15),  # north, east, depth, magnitude, time
    (0, 1, 2, 3, 4, 5), # year, month, day, hour, minute, second
    geoconv,
    nameconverter=nameconv,
)

# And save it to file for later use
io.write_event_table(evd, "data/events.txt")

print(f"The seismic event catalog has {len(evd)} records.")
print("The first event is:")
print(evd[0])

## Create a reference MT file

Next we will convert the moment tensor file. First, we inspect the file.

In [ ]:
! head ext/2022-007_Bloch-et-al_moment_tensor_catalog_correct_norm_v2.0.txt

The moment tensor catalog contains the components of the moment tensor. The format follows the Harvard convention used by the [Global CMT project](https://docs.generic-mapping-tools.org/latest/supplements/seis/meca.html) and documented e.g. in [GMT meca -Sm](https://docs.generic-mapping-tools.org/latest/supplements/seis/meca.html). The elements of the moment tensor in [dyne cm] units are described in columns 6 to 12 (0-based indexing).

For use in *relMT* we will next
* Convert the event time to a string that matches the event name of the catalog
* Convert from Harvard (R, $\theta$, $\phi$) to North-East-Down coordinates
* Convert from dyne cm to Newton meter (Nm)
* Write the moment tensor table

In [ ]:

# The external moment tensor catalog file
mtf = "ext/2022-007_Bloch-et-al_moment_tensor_catalog_correct_norm_v2.0.txt"

# Event date times in the catalog
evdatetimes = [ev.name for ev in evd.values()]

# Function to convert date and time strings to event names
def nameconverter(datestr, timestr):
    # Remove delimiters from date and time strings and concatenate
    date = datestr.replace("/", "")
    time = timestr.replace(":", "")
    
    # Times do not match exactly, so look up closest event time
    datetime = f"{date}{time}"
    evdatetime = utils.approx_time_lookup([datetime], evdatetimes)

    # Look up in single-element dict
    return evdatetime[datetime]

# Function to convert moment tensors from Harvard format to north-east-down in Nm
def mtconverter(mrr, mtt, mff, mrt, mrf, mtf, exp):
    # Convert to north, east, down ...
    mnn, mee, mdd, mne, mnd, med = mt.rtf2ned(mrr, mtt, mff, mrt, mrf, mtf)

    #  ... and from dyne cm to Nm
    fac = 10**exp * 1e-7
    return (mnn * fac, mee * fac, mdd * fac, mne * fac, mnd * fac, med * fac)

# Read the moment tensor catalog into a moment tensor dictionary
mtd = io.read_ext_mt_table(
    mtf,
    (6, 7, 8, 9, 10, 11),  # mrr, mtt, mff, mrt, mrf, mtf
    (0, 1), # Date strings
    evd,
    mtconverter,  # This mtconverter converts from Harvad to north-east-down convention
    12,  # exponent
    nameconverter,  # The names are used to associate MTs to events
)

# And save it to disk
io.write_mt_table(mtd, "data/reference_mts.txt")

print(f"The MT catalog has {len(mtd)} records, namely:")
print(", ".join(map(str, mtd.keys())))

## Conclusion
Let's have a look at the catalog and reference moment tensor file we just created. They now obey the *relMT* conventions, that is:
* units are in meter
* the axes point north, east and down, and
* a global event index (first column) is used to refer to the events

In [ ]:
! head data/events.txt

In [ ]:
! head data/reference_mts.txt